### Naïve Bayes

&nbsp;

The second lesson of generative learning is Naïve Bayes. NB is natural language processing 101. It is very naïve and simple but powerful, quite similar to logistic regression. The core part is merely calculating posterior probability from Bayes' theorem. We obtain p(y=label|x) from p(x|y=label)\*p(y=label)/p(x), where p(x) is written in the form of Σp(x|y=label)\*p(y=label). The form is quite consistent with another generative learning, Gaussian Discriminant Analysis. 

Feel free to check GDA in the following link

https://github.com/je-suis-tm/machine-learning/blob/master/gaussian%20discriminant%20analysis.ipynb


In [1]:
import os
import numpy as np
import pandas as pd
import nltk.tokenize
import nltk.corpus
import nltk.stem
import sklearn.model_selection
import sklearn.naive_bayes
import sklearn.feature_extraction.text
import scipy.special
import scipy.sparse
os.chdir('K:\ecole\github')

In [2]:
#convert text into a list of words
#we can use stemming and lemmatization to improve efficiency
#for instance, we have words walked,walking,walks
#with nltk package, we can revert all of them to walk
def text2list(text,stopword,lower=True,lemma=False,stemma=False):

    text_clean=text if lower==False else text.lower()
    
    #tokenize and remove stop words
    token=[i for i in nltk.tokenize.RegexpTokenizer(r'\w+').tokenize(text_clean) if i not in stopword]
    
    #lemmatization
    if lemma:
        text_processed=[nltk.stem.wordnet.WordNetLemmatizer().lemmatize(i) for i in token]
    else:
        text_processed=token
        
    #stemming
    if stemma:
        output=[nltk.stem.PorterStemmer().stem(i) for i in text_processed]
    else:
        output=text_processed
    
    #remove numbers as they are stopword as well
    for i in [ii for ii in output]:
        try:
            float(i)
            output.remove(i)
        except:
            pass
    
    return [i for i in output if i not in stopword]

In [3]:
#this is a function to calculate top 30 frequent words
#it would help us to determine which of those are stop words
#stop words refer to the words we dont take in account for naïve bayes
#such as you, i, and, of, the, an
#which are inevitable in both spam and ham
#it is a stop word helper
#we still need to filter out the words that dont belong in this list manually
#for each task, stop word may be different
#e.g. if every text you need to classify has a prefix like '[Confidential]'
#confidential becomes a stop word to be excluded
def get_stopword(output):
    
    vocabulary=sorted(list(set(output)))
    
    d={}
    stopword=[]
    
    for i in vocabulary:
        d[i]=output.count(i)
    
    #get top 30 frequent words
    for j in sorted(list(d.values()))[::-1][:30]:
        word=list(d.keys())[list(d.values()).index(j)]
        stopword.append(word)
        del d[word]
        
    return stopword


#there is another way to find stop words called kl divergence
#it calculates the distance from one distribution to another
#in layman’s terms, it gives us the impact of each word on classifications
#you can check Wikipedia for more details
# https://en.wikipedia.org/wiki/Kullback%E2%80%93Leibler_divergence
#it is also called mutual information in machine learning
#the actual formula is kl(p(x:y)||p(x)p(y))
#we want the divergence from xy joint distribution to xy independent distribution
#in scipy, u can use scipy.stats.entropy
#in sklearn, u can use sklearn.feature_selection.mutual_info_classif
def kl_distance(df):

    #concatenate all words together
    corpus=set([j for i in df['word'] for j in i])

    #probability of y is fixed
    #they are calculated before everything
    py0=len(df[df['spam']==0])/len(df)
    py1=len(df[df['spam']==1])/len(df)

    #calculate kl distance for each word
    kldict=[]
    for word in set(corpus):

        kldistance=0

        countx=[]

        #to make our life easier
        #we take multivariate approach
        #we dont calculate the frequency of a word
        for j in range(len(df)):
            if df['word'][j].count(word)>0:
                countx.append(1)
            else:
                countx.append(0)

        #using dataframe even though the performance is slower
        #it is easier to do slicing by logical expression
        data=pd.DataFrame()
        data['x']=countx
        data['y']=df['spam']

        px0=len(data[data['x']==0])/len(data)
        px1=len(data[data['x']==1])/len(data)

        #using loops to do sum
        for k in range(2):
            for l in range(2):
                prob_joint=len(data[data['y']==k][data['x']==l])/len(data)

                probx=px0 if l==0 else px1
                proby=py0 if k==0 else py1

                #we need to avoid 2 traps
                #zero division and logarithm zero
                try:
                    entropy=prob_joint*np.log(prob_joint/(probx*proby))
                except ZeroDivisionError:
                    entropy=0

                if prob_joint==0:
                    entropy=0

                kldistance+=entropy

        kldict+=[word,kldistance]

    #transform into dataframe and sort by kl distance in descending order
    output=pd.DataFrame()
    output['word']=kldict[0::2]
    output['kl distance']=kldict[1::2]

    output.sort_values(by='kl distance',ascending=False,inplace=True)

    output.reset_index(inplace=True,drop=True)

    return output

In [4]:
stopword=nltk.corpus.stopwords.words('english')+['u',
 'beyond',
 'within',
 'around',
 'would',
 'b',
 'c',
 'e',
 'f',
 'g',
 'h',
 'j',
 'k',
 'l',
 'n',
 'p',
 'q',
 'r',
 'u',
 'v',
 'w',
 'x',
 'z',
 'first']

In [5]:
#denote vocabulary as the list of all the words in english theoretically
#in reality, it is not feasible and economic to do so
#here, we just collect all words from all emails
#we gotta filter out those stop words as well
def get_vocabulary(output,stopword):
    
    vocabulary=sorted(list(set(output)))
    
    for i in vocabulary:
        if i in stopword:
            vocabulary.remove(i)
        
    return vocabulary    

In [6]:
#using box cox transformation to shift the distribution mean
#not recommended for naïve bayes
# https://en.wikipedia.org/wiki/Power_transform#Box%E2%80%93Cox_transformation
def box_cox(x,power):
       
    if power==0:
        return np.log(x)
    else:
        return (x**power-1)/power

In [7]:
#calculate p(x|y=classification) for multivariate naïve bayes
def multivariate_calc_prob(word,x_train,y_train,classification):
    
    #how many spam or ham from all emails
    num=list(y_train).count(classification)
    
    #check how many emails contain the given word
    corpus=[i.count(word) for i in x_train[y_train==classification]]
    freq=len(corpus)-corpus.count(0)
    
    #calculate p(x|y=classification)
    if freq!=0:
        p=freq/num
        
    else:
        
        #laplace smoothing
        #when the given word hasnt appeared in our training set yet
        #we gotta avoid the scenario 0/num
        #even if the given word has never appeared
        #it doesnt indicate it wont appear in the future
        #therefore, we gotta use laplace smoothing
        #see the following link for more details
        # https://en.wikipedia.org/wiki/Additive_smoothing
        
        #add 1 to the numerator and k to the denominator
        #where k is the dimension of x
        #in another word, how many possible values x can take
        #in binary classification, k=2
        #x either exists in the email or it doesnt
        p=(freq+1)/(num+2)

    return p


#multivariate event model follows a bernoulli distribution
#each word has two scenario
#it either appears in the email or not
def multivariate(sample,stopword):
    
    x_train,x_test,y_train,y_test= \
    sklearn.model_selection.train_test_split(sample['word'],sample['spam'],test_size=0.3)
    
    #calculate p(y)
    phi_y0=list(y_train).count(0)/len(y_train)
    phi_y1=1-phi_y0
    
    forecast=[]
    
    #assume all x are independent
    #each word has its own probability of occurrence in emails
    #the probability of occurrence of those words are not correlated
    #hence, p(x|y) for all x can be written as
    #the product of all probabilities of x
    #which is called chain rule in bayesian network
    #see the following link for details
    # https://en.wikipedia.org/wiki/Chain_rule_%28probability%29
    for i in x_test:
        px_y0,px_y1=1,1
        for j in i:
            if j not in stopword:
                px_y0*=multivariate_calc_prob(j,x_train,y_train,0)
                px_y1*=multivariate_calc_prob(j,x_train,y_train,1)
            else:
                pass
        
        #calculate p(y|x)
        py0_x=px_y0*phi_y0
        py1_x=px_y1*phi_y1
        
        #compare p(y=1|x) with p(y=0|x)
        #we take the larger one as for all generative learning algo
        #if the probabilities of spam and ham are equal
        #we d rather see it in inbox folder instead of spam folder
        p=0 if py0_x>=py1_x else 1
        forecast.append(p)
        
    return len(y_test[forecast==y_test])/len(y_test)*100

        

In [8]:
#calculate p(x|y) for multinomial naïve bayes
def multinomial_calc_prob(word,x_train,y_train,
                          classification,vocabulary,tfidf=False,
                          boxcox=False,boxcox_param=None):
    
    #get the word count of all spam/ham emails
    num=sum([len(i) for i in x_train[y_train==classification]])
       
    #get the frequency of given word in all spam/ham emails
    freq=sum([i.count(word) for i in x_train[y_train==classification]])
        
    #boxcox transformation
    #not recommended for naïve bayes
    if boxcox:
        num=box_cox(num,boxcox_param)
        freq=box_cox(freq,boxcox_param)

    if freq!=0:
        p=freq/num
        
    else:
        
        #laplace smoothing 
        #here x can take on v possible values
        #where v is the length of vocabulary list
        #we are assuming vocabulary list covers every word in english
        p=(freq+1)/(num+len(vocabulary))
        
    #tfidf is not recommended for naïve bayes
    if tfidf:
        
        tf=p
        
        #number of emails of spam/ham that contain the given word
        idf_den=len([i for i in x_train[y_train==classification] if word in i])
        
        #number of emails of spam/ham
        idf_num=len(x_train[y_train==classification])
        
        #laplace smoothing
        if idf_den==0:
            idf_num+=1
            idf_den+=1
        
        return tf*np.log(idf_num/idf_den)
        
    else:

        return p


#unlike multivariate, multinomial event model follows a multinomial distribution
#the frequency of each word is taken into consideration
#the formula is pretty much the same as multivariate
#except p(x|y) is different
def multinomial(sample,stopword,tfidf=False,
                boxcox=False,boxcox_param=None):
    
    x_train,x_test,y_train,y_test= \
    sklearn.model_selection.train_test_split(sample['word'],sample['spam'],test_size=0.3)
    
    vocabulary=[]
    for i in sample['word']:
        corpus=get_vocabulary(i,stopword)
        vocabulary+=corpus
    
    phi_y0=list(y_train).count(0)/len(y_train)
    phi_y1=1-phi_y0
    
    forecast=[]
    
    for i in x_test:
        px_y0,px_y1=1,1
        for j in i:
            if j not in stopword:
                px_y0*=multinomial_calc_prob(j,x_train,y_train,
                                             0,vocabulary,tfidf,
                                             boxcox,boxcox_param)
                px_y1*=multinomial_calc_prob(j,x_train,y_train,
                                             1,vocabulary,tfidf,
                                             boxcox,boxcox_param)
            else:
                pass
            
        py0_x=px_y0*phi_y0
        py1_x=px_y1*phi_y1
        
        p=0 if py0_x>=py1_x else 1
        forecast.append(p)
        
    return len(y_test[forecast==y_test])/len(y_test)*100

In [9]:
#create multivariate vocabulary vector for sklearn
#this is the authenticate way in naïve bayes
#but using pandas is more convenient
#which is why our own implementation is kinda different
def multivariate_vector(x,y,vocabulary):
    
    #CRUCIAL!!!!
    #train_test_split would shuffle the training set
    #the shuffled index would cause a problem later
    x.reset_index(inplace=True,drop=True)
    y.reset_index(inplace=True,drop=True)
    
    #we create a m*n matrix
    #where m denotes the number of emails(rows)
    #n denotes the number of words in vocabulary(columns)
    #the value denotes if word[n] appears in email[m]
    #if so, we set the value to 1
    df_multivariate=pd.DataFrame()
    for i in vocabulary:
        document_term_matrix=[]
        for j in x.index:
            if i in x[j]:
                document_term_matrix.append(1)
            else:
                document_term_matrix.append(0)
                
        df_multivariate[i]=document_term_matrix
        
    df_multivariate['real y']=y
    
    return df_multivariate


#create multinomial vocabulary vector for sklearn
def multinomial_vector(x,y,vocabulary):
    
    #CRUCIAL!!!!!
    x.reset_index(inplace=True,drop=True)
    y.reset_index(inplace=True,drop=True)
    
    #we create a m*n matrix
    #where m denotes the number of emails(rows)
    #n denotes the number of words in vocabulary(columns)
    #the value denotes the frequency of word[n] in email[m]
    df_multinomial=pd.DataFrame()
    for i in vocabulary:
        document_term_matrix=[]
        for j in x.index:
            document_term_matrix.append(x[j].count(i))
                
        df_multinomial[i]=document_term_matrix
        
    df_multinomial['real y']=y
    
    return df_multinomial


#using the official sklearn package
def sklearn_off(sample,stopword,method,nbmethod):
    
    vocabulary=[]
    for i in sample['word']:
        corpus=get_vocabulary(i,stopword)
        vocabulary+=corpus
    
    x_train,x_test,y_train,y_test= \
    sklearn.model_selection.train_test_split(sample['word'],sample['spam'],test_size=0.3)

    mn_train=method(x_train,y_train,vocabulary)
    mn_test=method(x_test,y_test,vocabulary)
    
    model=nbmethod().fit(pd.concat([mn_train[i] for i in vocabulary], \
                                      axis=1),mn_train['real y'])
    
    forecast=model.predict(pd.concat([mn_test[i] for i in vocabulary], \
                                      axis=1))
    
    return len(y_test[forecast==y_test])/len(y_test)*100

In [10]:
#quick bernoulli
def sklearn_ber(sample):

    x_train,x_test,y_train,y_test= \
    sklearn.model_selection.train_test_split(sample['word'],sample['spam'],test_size=0.3)
    
    #create vocabulary
    vec=sklearn.feature_extraction.text.CountVectorizer(binary=True)
    vec.fit(x_train.apply(lambda x:' '.join(x)))
    matrix=vec.transform(x_train.apply(lambda x:' '.join(x)))
    
    #train
    clf=sklearn.naive_bayes.BernoulliNB()
    clf.fit(matrix,y_train)

    #using the same vocabulary
    matrix=vec.transform(x_test.apply(lambda x:' '.join(x)))

    return clf.score(matrix,y_test)*100


#quick multinomial
def sklearn_mul(sample,vecmethod='default',
                boxcox=False,boxcox_param=None):

    x_train,x_test,y_train,y_test= \
    sklearn.model_selection.train_test_split(sample['word'],sample['spam'],test_size=0.3)
    
    #using tfidf is not recommended for naïve bayes
    #more details about tfidf can be checked from the link
    # http://tfidf.com/
    #
    if vecmethod=='default':
        vec=sklearn.feature_extraction.text.CountVectorizer()
    else:
        vec=sklearn.feature_extraction.text.TfidfVectorizer()
        
    #create vocabulary
    vec.fit(x_train.apply(lambda x:' '.join(x)))
    matrix=vec.transform(x_train.apply(lambda x:' '.join(x)))
    
    #using box cox transformation
    #not recommended for naïve bayes
    if boxcox:
        dense=scipy.special.boxcox(matrix.todense(),boxcox_param)
        matrix=scipy.sparse.csr_matrix(dense,matrix.shape)
    
    #train
    clf=sklearn.naive_bayes.MultinomialNB()
    clf.fit(matrix,y_train)
    
    #using the same vocabulary
    matrix=vec.transform(x_test.apply(lambda x:' '.join(x)))

    return clf.score(matrix,y_test)*100

In [11]:
#the raw data really comes from my email
df=pd.read_csv('spam.csv')

In [12]:
#tokenization
tokens=[]
for i in df['text'].tolist():
    tokens.append(text2list(i,stopword,lower=True,lemma=True))
            
df['word']=tokens

In [13]:
#to review stopword
print(get_stopword([j for i in tokens for j in i]))

['settlement', 'please', 'class', 'google', 'data', 'service', 'new', 'information', 'privacy', 'fund', 'term', 'app', 'payment', 'may', 'member', 'policy', 'account', 'trip', 'update', 'claim', 'use', 'application', 'oculus', 'reddit', 'time', 'agreement', 'com', 'court', 'updated', 'check']


In [14]:
#to check each word's contribution to the result
print(kl_distance(df))

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:79: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:87: RuntimeWarning: divide by zero encountered in log
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:87: RuntimeWarning: invalid value encountered in double_scalars


              word  kl distance
0           office     0.188020
1             team     0.155811
2          updated     0.132911
3        following     0.126903
4          payment     0.126903
5           invite     0.126903
6           family     0.126903
7        organized     0.126903
8             love     0.126903
9           number     0.126903
10         privacy     0.111192
11           thank     0.104636
12          change     0.104636
13            note     0.102175
14        everyone     0.098905
15       organizer     0.098905
16          friend     0.098905
17         neither     0.098905
18           quick     0.098905
19      eventbrite     0.098905
20         gabriel     0.098905
21        organize     0.098905
22          amount     0.098905
23           might     0.098905
24       traveling     0.098905
25       attending     0.098905
26         student     0.098905
27            name     0.098905
28              uk     0.098905
29             pay     0.098905
...     

In [15]:
#apart from using vocabulary
#we can also use len(text) and % of punctuations as features

#try ten times and compare the result of self implementation and sklearn
accuracies=[]
for i in range(10):
        
    accuracies.append(multivariate(df,stopword))
    accuracies.append(multinomial(df,stopword))
    accuracies.append(sklearn_ber(df))
    accuracies.append(sklearn_mul(df))

In [16]:
for j,k in enumerate(['multivariate implementation','multinomial implementation',
                        'multivariate sklearn','multinomial sklearn']):
    print('{} accuracy: {}%'.format(k,np.mean(accuracies[j::4])))

multivariate implementation accuracy: 74.54545454545456%
multinomial implementation accuracy: 81.81818181818183%
multivariate sklearn accuracy: 70.9090909090909%
multinomial sklearn accuracy: 85.45454545454547%


In [17]:
#surprisingly the accuracy of my implementation is actually higher than sklearn
#but it is a huge tradeoff for time
#i would definitely use sklearn instead
#btw, my sample size is very small
#the result is definitely biased
#it heavily depends on how train and test sets are split